<a href="https://colab.research.google.com/github/devansong/network_workshop/blob/master/R_Wildlife_Trade_Network_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# URI Illegal Wildlife Trafficking: R workshop in network analysis

## Setup

This section installs and loads all the packages we need, and downloads the CITES dataset using the [citesdb](https://github.com/ropensci/citesdb) package.  We'll call this dataset `master`.

This takes approximately 10-15 minutes to run.

In [ ]:
install.packages("igraph")
devtools::install_github("ropensci/citesdb")

library(igraph)
library(tidyverse)
library(citesdb)

# Collect dataset, and remove NA's from the Importer/Exporter columns
cites_db_download()
master <- cites_shipments() %>% 
    collect()
master$Importer[is.na(master$Importer)] <- ""
master$Exporter[is.na(master$Exporter)] <- ""

## Test
Before we begin the class, let's make sure we can run some simple code and plot a graph. 
Click on each cell of code and press "shift + enter"

In [ ]:
x <- c(1,3,6,9,12)
y <- c(1.5,2,7,8,15)
plot(
    x, y, 
    xlab="x axis", 
    ylab="y axis", 
    main="my plot",
    ylim=c(0,20),
    xlim=c(0,15), 
    pch=19,
    col="blue",
    cex=3
)

If you see a simple scatter plot with 5 data points, you have successfully run R code!

## Explore the CITES dataset

In [ ]:
str(master)

In [ ]:
head(master)

In [ ]:
min(master$Year) 

In [ ]:
max(master$Year) 

In [ ]:
table(master$Class) 

In [ ]:
sort(table(master$Class))

In [ ]:
plot(sort(table(master$Class)))

## Functions

In [ ]:
create_network <- function(data){
  
  sources <- data %>%
    distinct(Exporter) %>%
    rename(label = Exporter) #creates data frame of all countries of origin 
  
  destinations <- data %>%
    distinct(Importer) %>%
    rename(label = Importer)#creates data frame of all countries importing 
  
  nodes <- full_join(sources, destinations, by = "label") #creates data frame of all unique countries in your dataset 
  nodes <- nodes %>% rowid_to_column("id") #creates numerical id for each unique country 

# this next section of code creates a tibble with export id, import id, and weight by number of shipments ~~~~  
  per_route <- data %>%  
    group_by(Exporter, Importer) %>%
    summarise(weight = n()) %>% #number of observations in the current group
    ungroup() #removes grouping 
  edges <- per_route %>% 
    left_join(nodes, by = c("Exporter" = "label")) %>% 
    rename(from = id)
  edges <- edges %>% 
    left_join(nodes, by = c("Importer" = "label")) %>% 
    rename(to = id)
  edges <- select(edges, from, to, weight)
#~~~~~~~~~#
  
  #creates a directed igraph network from the edges and nodes defined above   
  net <- graph_from_data_frame(d=edges, vertices=nodes, directed=T) 
}

In [ ]:
plot_network <- function(igraph_network, title="trade network", weight_denom=100, layout=layout_with_mds){
  E(igraph_network)$width <- E(igraph_network)$weight/weight_denom
  l <- layout(igraph_network)
  plot(
    igraph_network,
    edge.color= rgb(70/255, 130/255, 180/255, 0.6),
    edge.arrow.size=.001, 
    vertex.size = 7, 
    vertex.color="gold", #(65,105,225)
    vertex.label = V(igraph_network)$id,
    vertex.label.cex = 0.6, 
    vertex.label.color = "black", 
    rescale=T, 
    layout=l*3.0, 
    main=title) 
}

## Create and visualize network

In [ ]:
master_network <-create_network(master) 

In [ ]:
plot_network(master_network, weight_denom=40000, layout = layout_randomly)

In [ ]:
list.vertex.attributes(master_network)
which(V(master_network)$label == "CZ")
which(V(master_network)$label == "US")
which(V(master_network)$label == "")
# Remove CZ (seems problematic) and empty vertex 
master_CZ_rem <- delete_vertices(master_network, c(220))
master_CZ_rem <- delete_vertices(master_network, c(43))

In [ ]:
master_network <- master_CZ_rem
s1 <- subgraph.edges(master_network, E(master_network)[E(master_network)$weight>2000], del=F)
s2 <- delete_vertices(s1, degree(s1, mode = "in")==0)
plot_network(s2, weight_denom=40000, layout = layout_randomly)

In [ ]:
plot_network(s2, weight_denom=40000, layout = layout_with_fr)


In [ ]:
plot_network(s2, weight_denom=40000, layout = layout_with_mds)


## Create 2 subsets of the data, create networks, and visually compare them

In [ ]:
Amp_1992 <- subset(master, Class == "Amphibia" & Year == "1992") #Creates subset of 1992 amphibians
Amp_2012 <- subset(master, Class == "Amphibia" & Year == "2012") #Creates subset of 2012 amphibians

In [ ]:
net1<-create_network(Amp_1992) #Create network for subset 1
net2 <-create_network(Amp_2012) #Create network for subset 2

In [ ]:
par(mfrow=c(1,2))
plot_network(net1, title="1992", weight_denom=5, layout= layout_on_sphere)
plot_network(net2, title="2012", weight_denom=5, layout= layout_on_sphere)

## Cluster analysis

In [ ]:
par(mfrow=c(1,2))
cfg <- cluster_fast_greedy(as.undirected(net1))
plot(cfg, as.undirected(net1)) 
title(main = "Amphibians 1992")
cfg <- cluster_fast_greedy(as.undirected(net2))
plot(cfg, as.undirected(net2)) 
title(main = "Amphibians 2012")

## Exercise


Modify the above 4 sections of code to compare two

## Network and node descriptive stats

In [ ]:
ecount(net1)/(vcount(net1)*(vcount(net1)-1)) #for a directed network
ecount(net2)/(vcount(net2)*(vcount(net2)-1)) #for a directed network

In [ ]:
reciprocity(net1)
reciprocity(net2)

In [ ]:
transitivity(net1, type="global")
transitivity(net2, type="global")

In [ ]:
diameter(net1, directed=F, weights=NA)
diameter(net2, directed=F, weights=NA)

In [ ]:
deg1 <- degree(net1, mode="all") #NODE METRICS ************
plot(net1, vertex.size=deg1*3, edge.arrow.size=0.1)
hist(deg1, breaks=1:vcount(net1)-1, main="Histogram of node degree")

In [ ]:
deg2 <- degree(net2, mode="all") #NODE METRICS ************
plot(net2, vertex.size=deg2*3, edge.arrow.size=0.1)
hist(deg2, breaks=1:vcount(net2)-1, main="Histogram of node degree")

## References
Ross, Noam, Evan A. Eskew, and Nicolas Ray. 2019. citesdb: A high-performance database of shipment-level CITES trade data. R package v0.2.0. EcoHealth Alliance: New York, NY. https://github.com/ropensci/citesdb. doi:10.5281/zenodo.2630836

UNEP-WCMC (Comps.) 2019. Full CITES Trade Database Download. Version 2019.2. CITES Secretariat, Geneva, Switzerland. Compiled by UNEP-WCMC, Cambridge, UK. Available at: https://trade.cites.org.